<a href="https://colab.research.google.com/github/pksvv/lang-transformers/blob/main/TF_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Data Preprocessing
* TF input pipeline
* Building and Training
* Making Predictions

# Dataset

https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data

In [2]:
pip install kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [4]:
ls -alrth ~

total 64K
-r-xr-xr-x 1 root root  254 Jan  1  2000 .tmux.conf*
-r-xr-xr-x 1 root root 1.2K Jan  1  2000 .bashrc*
-rw-r--r-- 1 root root  148 Aug 17  2015 .profile
drwxr-xr-x 3 root root 4.0K Dec  3 14:33 .gsutil/
drwxr-xr-x 5 root root 4.0K Dec 16 14:10 .ipython/
drwx------ 1 root root 4.0K Dec 16 14:10 .local/
drwx------ 2 root root 4.0K Dec 16 14:10 .jupyter/
drwxr-xr-x 1 root root 4.0K Dec 16 14:10 .config/
drwxr-xr-x 4 root root 4.0K Dec 16 14:11 .npm/
drwxr-xr-x 1 root root 4.0K Dec 16 14:13 .cache/
drwxr-xr-x 1 root root 4.0K Dec 31 08:07 ../
drwxr-xr-x 2 root root 4.0K Dec 31 08:07 .keras/
drwxr-xr-x 2 root root 4.0K Dec 31 08:08 .kaggle/
drwx------ 1 root root 4.0K Dec 31 08:08 ./


In [9]:
mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [10]:
mv ~/kaggle.json ~/.kaggle/kaggle.json

In [11]:
ls ~/.kaggle

kaggle.json


In [12]:
import kaggle

In [13]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [14]:
api = KaggleApi()

In [15]:
api.authenticate()

In [16]:
api.competition_download_file('sentiment-analysis-on-movie-reviews',
                              'test.tsv.zip',
                              path='./')

api.competition_download_file('sentiment-analysis-on-movie-reviews',
                              'train.tsv.zip',
                              path='./')

100%|██████████| 494k/494k [00:00<00:00, 54.7MB/s]

100%|██████████| 1.28M/1.28M [00:00<00:00, 97.7MB/s]

In [21]:
ls ./

sample_data/  test.tsv  test.tsv.zip  train.tsv  train.tsv.zip


In [20]:
import zipfile

with zipfile.ZipFile('./test.tsv.zip','r') as zipref:
  zipref.extractall('./')

with zipfile.ZipFile('./train.tsv.zip','r') as zipref:
  zipref.extractall('./')

In [22]:
import pandas as pd

df = pd.read_csv('./train.tsv',sep='\t')

df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [23]:
df['Phrase'][0]

'A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .'

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PhraseId    156060 non-null  int64 
 1   SentenceId  156060 non-null  int64 
 2   Phrase      156060 non-null  object
 3   Sentiment   156060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.8+ MB
